In [8]:
import pandas as pd
import random 
from string import ascii_uppercase

def random_string_generator(n, k):
    random_strings = []
    for _ in range(n):
        random_strings.append(''.join(random.choice(ascii_uppercase) for i in range(k)))
    return random_strings

def random_ints_generator(n, max_val=1000):
    return [int(max_val*random.random()) for i in range(n)]

N_TRAIN = 1000
data = pd.DataFrame(columns=['image_filepath','mask_filepath','width','height','image_id'])
data['image_filepath'] = random_string_generator(N_TRAIN, 10)
data['mask_filepath'] = random_string_generator(N_TRAIN, 10)
data['width'] = random_ints_generator(N_TRAIN, 100)
data['height'] = random_ints_generator(N_TRAIN, 100)
data['image_id'] = random_ints_generator(N_TRAIN, 10000)
display(data.head())
data.to_csv('/mnt/ml-team/dsb_2018/kuba/mocked_data/metadata_train.csv',index=None)

N_TEST = 100
data = pd.DataFrame(columns=['image_filepath','width','height','image_id'])
data['image_filepath'] = random_string_generator(N_TEST, 10)
data['width'] = random_ints_generator(N_TEST, 100)
data['height'] = random_ints_generator(N_TEST, 100)
data['image_id'] = random_ints_generator(N_TEST, 10000)
display(data.head())
data.to_csv('/mnt/ml-team/dsb_2018/kuba/mocked_data/metadata_test.csv',index=None)

,image_filepath,mask_filepath,width,height,image_id
0,BSUHQRKZCX,AWUFEDNRLG,36,5,4418
1,DSEKLIKDYA,KBIKDWBBSY,57,86,4432
2,MVGMGZERTC,GKPJBIXIZN,64,5,6253
3,SAYNDKOKAQ,BXLKQCSYFG,58,90,6964
4,BLGDPUMMYT,PKJYGJJRDT,63,55,9055


,image_filepath,width,height,image_id
0,PRUXARGXWM,84,47,3285
1,WOMQEDVSJU,77,40,7385
2,NEFHIXHUCB,38,17,3784
3,BHPIVMCSCL,25,37,4146
4,NVGNSHZAXE,82,68,9652
